In [0]:
filePath = "dbfs:/FileStore/GlobalRetail/bronze_layer/transaction/transaction_snappy.parquet"
df = spark.read.parquet(filePath)

In [0]:
from pyspark.sql.functions import to_timestamp,col
new_df = df.withColumn("transaction_date", to_timestamp(col("transaction_date")))
new_df.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- transaction_date: timestamp (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- store_type: string (nullable = true)



In [0]:
display(new_df.limit(10))

transaction_id,customer_id,product_id,quantity,total_amount,transaction_date,payment_method,store_type
TRX000001,802,425,1,363.4,2020-07-27T00:00:00.000+0000,Debit Card,Physical Store
TRX000002,858,280,6,758.18,2022-08-10T00:00:00.000+0000,Credit Card,Physical Store
TRX000003,658,694,9,748.66,2020-05-22T00:00:00.000+0000,Bank Transfer,Online
TRX000004,516,930,4,933.78,null,Bank Transfer,Physical Store
TRX000005,368,104,10,137.28,2022-06-24T00:00:00.000+0000,PayPal,Physical Store
TRX000006,606,409,6,556.88,2020-10-14T00:00:00.000+0000,Credit Card,Online
TRX000007,535,487,7,246.29,2023-08-19T00:00:00.000+0000,Credit Card,Physical Store
TRX000008,87,630,8,-416.82,2020-04-18T00:00:00.000+0000,Credit Card,Physical Store
TRX000009,383,617,10,405.05,2021-02-13T00:00:00.000+0000,Cash,Physical Store
TRX000010,725,938,1,410.03,2020-03-18T00:00:00.000+0000,PayPal,Online


In [0]:
from pyspark.sql.functions import current_timestamp
df_new = new_df.withColumn("ingestion_timestamp", current_timestamp())
display(df_new.limit(10))

transaction_id,customer_id,product_id,quantity,total_amount,transaction_date,payment_method,store_type,ingestion_timestamp
TRX000001,802,425,1,363.4,2020-07-27T00:00:00.000+0000,Debit Card,Physical Store,2025-04-29T11:50:23.268+0000
TRX000002,858,280,6,758.18,2022-08-10T00:00:00.000+0000,Credit Card,Physical Store,2025-04-29T11:50:23.268+0000
TRX000003,658,694,9,748.66,2020-05-22T00:00:00.000+0000,Bank Transfer,Online,2025-04-29T11:50:23.268+0000
TRX000004,516,930,4,933.78,null,Bank Transfer,Physical Store,2025-04-29T11:50:23.268+0000
TRX000005,368,104,10,137.28,2022-06-24T00:00:00.000+0000,PayPal,Physical Store,2025-04-29T11:50:23.268+0000
TRX000006,606,409,6,556.88,2020-10-14T00:00:00.000+0000,Credit Card,Online,2025-04-29T11:50:23.268+0000
TRX000007,535,487,7,246.29,2023-08-19T00:00:00.000+0000,Credit Card,Physical Store,2025-04-29T11:50:23.268+0000
TRX000008,87,630,8,-416.82,2020-04-18T00:00:00.000+0000,Credit Card,Physical Store,2025-04-29T11:50:23.268+0000
TRX000009,383,617,10,405.05,2021-02-13T00:00:00.000+0000,Cash,Physical Store,2025-04-29T11:50:23.268+0000
TRX000010,725,938,1,410.03,2020-03-18T00:00:00.000+0000,PayPal,Online,2025-04-29T11:50:23.268+0000


In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/global_retail_bronze.db/bronze_transactions", recurse=True)

Out[5]: True

In [0]:
# we want to save it as delta lake table because we are creating data lake house architecture (delta lake is underlying table format)
# delta lake provides you the capability to do crud operations and acid transactions
spark.sql("use global_retail_bronze")
try:
    df_new.write.format("delta").mode("append").saveAsTable("bronze_transactions")
except Exception as e:
    print(f"Error occurred: {str(e)}")

In [0]:
%sql
show tables

database,tableName,isTemporary
global_retail_bronze,bronze_customer,false
global_retail_bronze,bronze_products,false
global_retail_bronze,bronze_transactions,false


In [0]:
# import datetime

# archive_folder = 'dbfs:/FileStore/GlobalRetail/bronze_layer/transaction/archive/'
# archive_file_path = archive_folder+'_'+datetime.datetime.now().strftime("%Y%m%d%H%M%s")
# dbutils.fs.mv(filePath, archive_file_path)
# print(archive_file_path)